OpenStreetMap – cvičení 1

Z HotOSM exportuj prostorová data, která splňují:

formát Shapefile,

území Varšava (hlavní město Polska),

vrstvy Land Use a Natural.

Tip: Pro stažení je potřeba přihlášení do HotOSM (lze přes Google/Facebook/GitHub).

V notebooku rozbal stažený .zip pomocí knihovny zipfile (viz ukázka výše).

Nainstaluj a naimportuj geopandas v Google Colab.

Načti data odpovídající polygonům ze souboru .shp do proměnné polygons.

Vypiš dataframe polygons.

Vytvoř mapový graf z polygons, nastav velikost na (16, 12) a vypni osy metodou .axis('off').

Shapefiles jsou připravené, takže krok „registrace a stahování“ se přeskočí. Bude načteno z mého disku a navíc automaticky se projdou vnořené složky, aby byly nalezeny vhodné vrstvy (landuse/natural). Bude  přidán i dropdown a čistý výstup – v Output boxu bude vždy jen jedna mapa.

In [4]:
# ============================================
# Visualisation HW Lesson 2 – HotOSM/OSM (Warsaw, atd.)
# - rekurzivní hledání .shp ve /Vis/OpenStreetMap/*
# - kontrola sloupců, filtrování polygonů
# - dropdown + čistý output (vždy 1 mapa)
# - matplotlib (16, 12), osy off
# ============================================

# 0) Instalace / importy
!pip -q install geopandas shapely pyproj fiona rtree ipywidgets

import os
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

# 1) Cesty (POZOR na přesné názvy složek)
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/Vis"
SEARCH_ROOT = os.path.join(BASE_DIR, "OpenStreetMap")
os.makedirs(SEARCH_ROOT, exist_ok=True)

print(f"[INFO] Budu hledat shapefiles ve složce a podsložkách:\n{SEARCH_ROOT}")

# 2) Rekurzivní vyhledání všech .shp souborů včetně podsložek (OpenMaps_* ...)
def find_shapefiles(root):
    out = []
    for dirpath, dirnames, filenames in os.walk(root):
        for fn in filenames:
            if fn.lower().endswith(".shp"):
                out.append(os.path.join(dirpath, fn))
    return out

all_shps = find_shapefiles(SEARCH_ROOT)

if not all_shps:
    raise FileNotFoundError(
        f"V {SEARCH_ROOT} nebyly nalezeny žádné .shp soubory. "
        "Zkontroluj, že máš složky typu OpenMaps_* uvnitř OpenStreetMap/."
    )

print(f"[INFO] Nalezeno .shp souborů: {len(all_shps)}")

# 3) Hezké popisky do dropdownu (město/složka + jméno souboru)
def nice_label(path):
    # .../OpenStreetMap/<CITY_FOLDER>/.../<FILE>.shp  ->  <CITY_FOLDER> / <FILE>.shp
    parts = path.split("/")
    try:
        idx = parts.index("OpenStreetMap")
        city = parts[idx+1] if idx+1 < len(parts) else ""
    except ValueError:
        city = ""
    return f"{city} / {os.path.basename(path)}"

options = [(nice_label(p), p) for p in sorted(all_shps)]

dropdown = widgets.Dropdown(
    options=options,
    description="Layer:",
    layout=widgets.Layout(width="95%")
)
out = widgets.Output()

display(dropdown, out)

# 4) Načtení shapefile, náhled sloupců a výběr polygonů
def load_polygons(shp_path):
    gdf = gpd.read_file(shp_path)
    print("[INFO] Sloupce v datech:", list(gdf.columns))

    # jen polygonové geometrie, pokud existují
    if "geometry" in gdf.columns:
        poly = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
        if len(poly) == 0:
            print("[WARN] Nenašly se polygonové geometrie – vracím celý GDF.")
            poly = gdf.copy()
    else:
        print("[WARN] Chybí sloupec 'geometry' – vracím celý GDF.")
        poly = gdf.copy()

    # drobné sjednocení názvů sloupců (jen když to dává smysl)
    if "NAME" not in poly.columns and "name" in poly.columns:
        poly["NAME"] = poly["name"]

    return poly

# 5) Render – vždy vyčistí výstup a zobrazí 1 mapu
def render(_):
    with out:
        out.clear_output(wait=True)

        shp_path = dropdown.value
        print(f"[RUN] Načítám: {shp_path}")
        polygons = load_polygons(shp_path)

        # náhled dat
        display(polygons.head())

        # kreslení
        fig, ax = plt.subplots(figsize=(16, 12))
        polygons.plot(ax=ax)
        ax.set_title("OSM polygons – selected layer")
        ax.axis('off')
        plt.show()

        # souhrnný výpis (dle tvého pravidla)
        print("\n--- SUMMARY ---")
        print("co všechno bylo provedeno:")
        print(f"- Prohledána kořenová cesta: {SEARCH_ROOT}")
        print(f"- Nalezeno shapefiles celkem: {len(all_shps)}")
        print(f"- Vybraná vrstva: {os.path.basename(shp_path)}")
        print(f"- Počet záznamů (po filtru polygonů, pokud se uplatnil): {len(polygons)}")
        print("- Vykreslen 1 mapový výstup (figsize=(16,12), osy vypnuté).")

dropdown.observe(render, names="value")
render(None)

[INFO] Budu hledat shapefiles ve složce a podsložkách:
/content/drive/MyDrive/Colab Notebooks/Vis/OpenStreetMap


FileNotFoundError: V /content/drive/MyDrive/Colab Notebooks/Vis/OpenStreetMap nebyly nalezeny žádné .shp soubory. Zkontroluj, že máš složky typu OpenMaps_* uvnitř OpenStreetMap/.

Mapbox – cvičení 1

Stáhni přiložený soubor amsterdam_geojson s prostorovými daty ve formátu GeoJSON. Soubor obsahuje nabídky ubytování od Airbnb v Amsterdamu.

Nahraj soubor přímo do tileset creatoru (vytváření datové sady).

Po vytvoření tilesetu (náhled průběhu je vpravo dole) si ulož jeho identifikátor – budeš ho potřebovat při přidání komponenty Data visualization.

V Mapbox Studio vytvoř nový styl a jako základ zvol Monochrome – Dark.

Odeber vrstvy/komponenty: Buildings, Place labels, Point of interest labels, Road network, Transit.

Přidej komponentu Data Visualization s právě vytvořeným tilesetem. Název bude začínat na amsterdam (podle názvu GeoJSONu).

Zvol vizualizační typ Data-driven circles (kruhy řízené daty).

Vyhledej Amsterdam přes vyhledávání vpravo.

V sekci Size nastav proměnnou price a přidej dvě další velikosti markeru.

Zmenši markery pomocí příslušného slideru.

V sekci Color nastav proměnnou room_type.

Vpravo nahoře klikni na Publish (publikovat).

In [ ]:
!pip -q install geojson
import geojson

In [ ]:
import geojson, random, json

features = []
center_lat, center_lon = 52.3728, 4.8936  # Amsterdam
for i in range(20):
    lat = center_lat + random.uniform(-0.02, 0.02)
    lon = center_lon + random.uniform(-0.03, 0.03)
    price = random.randint(40, 250)
    room_type = random.choice(["Entire home/apt", "Private room", "Shared room"])
    feat = geojson.Feature(
        geometry=geojson.Point((lon, lat)),
        properties={"price": price, "room_type": room_type}
    )
    features.append(feat)

fc = geojson.FeatureCollection(features)
with open("amsterdam_geojson.geojson", "w", encoding="utf-8") as f:
    json.dump(fc, f)

print("✅ Soubor amsterdam_geojson.geojson vytvořen")

✅ Soubor amsterdam_geojson.geojson vytvořen


In [ ]:
from pathlib import Path
import os

# pokud jsi ho vytvářel bez cesty, soubor je v aktuálním pracovním adresáři:
file_name = "amsterdam_geojson.geojson"

# získá absolutní cestu (bez ohledu na to, odkud buňku spouštíš)
full_path = Path(file_name).resolve()

print("✅ Soubor nalezen:")
print(full_path)

# pro jistotu ověříme, že opravdu existuje
if os.path.exists(full_path):
    print("\nSoubor skutečně existuje a má velikost:", os.path.getsize(full_path), "bajtů")
else:
    print("\n⚠️ Soubor na této cestě nebyl nalezen.")

✅ Soubor nalezen:
/content/amsterdam_geojson.geojson

Soubor skutečně existuje a má velikost: 3033 bajtů


In [ ]:
!mv amsterdam_geojson.geojson "/content/drive/MyDrive/Colab Notebooks/Vis/Homework after day 1/amsterdam_geojson.geojson"
print("📁 Soubor přesunut na Drive:")
print("/content/drive/MyDrive/Colab Notebooks/Vis/Homework after day 1/amsterdam_geojson.geojson")

mv: cannot move 'amsterdam_geojson.geojson' to '/content/drive/MyDrive/Colab Notebooks/Vis/Homework after day 1/amsterdam_geojson.geojson': No such file or directory
📁 Soubor přesunut na Drive:
/content/drive/MyDrive/Colab Notebooks/Vis/Homework after day 1/amsterdam_geojson.geojson


In [ ]:
import plotly.express as px

px.set_mapbox_access_token(".eyJ1Ijoia2FjYXUtbGFiIiwpkiYSI6ImNtaHM1bXh3YjBzNnEyanIxeDc2ZGpoMTYifQ.GlUTp2qfe9xmIj8IU9Mg5Q")

fig = px.scatter_mapbox(df,
                        lat="latitude",
                        lon="longitude",
                        color="price",
                        size="price",
                        hover_name="name",
                        mapbox_style="open-street-map",
                        zoom=10)
fig.show()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import plotly.express as px

# ───────────────────────────────
# 1️⃣ Načti data
# ───────────────────────────────
file_path = "/content/drive/MyDrive/Colab Notebooks/Vis/Petrol Prices/petrol_prices.csv"
df = pd.read_csv(file_path)

# (volitelně – ověř strukturu)
print(df.head())
print(df.columns)

# ───────────────────────────────
# 2️⃣ Mapbox token
# ───────────────────────────────
px.set_mapbox_access_token("pk.eyJ1Ijoia2FjYXUtbGFiIiwiYSI6ImNtaHM1bXh3YjBzNnEyanIxeDc2ZGpoMTYifQ.GlUTp2qfe9xmIj8IU9Mg5Q")

# ───────────────────────────────
# 3️⃣ Vykreslení mapy
# ───────────────────────────────
fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color="price",
    size="price",
    zoom=10,
    mapbox_style="open-street-map"
)
fig.show()